In [1]:
import numpy as np
import pandas as pd
import requests
import wikipedia as wp

C:\Users\junta\Anaconda2\envs\py3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Get the html source from wikipedia: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header=0)[0]
# Delete rows with borough of 'Not assigned'
df = df[~df.Borough.str.contains("Not assigned")]
# Assign the name of borough to neighbourhood if neighbourhood is not assigned
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
df.head() 

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [3]:
# Group by postcode and then borough, join the neighbourhoods together into one column with delimater coma.
df_grouped = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df_grouped.shape

(103, 3)

In [40]:
df_grouped['Latitude'] = None
df_grouped['Longitude'] = None

In [41]:
# Get latitudes and longitudes from google maps geocoding api
import googlemaps
google_key = googlemaps.Client(key = 'xxxxxxx')
for i in range(len(df_grouped)):
    g = google_key.geocode('{}, Toronto, Ontario'.format(df_grouped.loc[i, 'Postcode']))
    try:
        lat = g[0]['geometry']['location']['lat']
        lng = g[0]['geometry']['location']['lng']
        df_grouped.loc[i, 'Latitude'] = lat
        df_grouped.loc[i, 'Longitude'] = lng
        print(i, 'finished!')
    except:
        print(i, 'error!')

0 finished!
1 finished!
2 finished!
3 finished!
4 finished!
5 finished!
6 finished!
7 finished!
8 finished!
9 finished!
10 finished!
11 finished!
12 finished!
13 finished!
14 finished!
15 finished!
16 finished!
17 finished!
18 finished!
19 finished!
20 finished!
21 finished!
22 finished!
23 finished!
24 finished!
25 finished!
26 finished!
27 finished!
28 finished!
29 finished!
30 finished!
31 finished!
32 finished!
33 finished!
34 finished!
35 finished!
36 finished!
37 finished!
38 finished!
39 finished!
40 finished!
41 finished!
42 finished!
43 finished!
44 finished!
45 finished!
46 finished!
47 finished!
48 finished!
49 finished!
50 finished!
51 finished!
52 finished!
53 finished!
54 finished!
55 finished!
56 finished!
57 finished!
58 finished!
59 finished!
60 finished!
61 finished!
62 finished!
63 finished!
64 finished!
65 finished!
66 finished!
67 finished!
68 finished!
69 finished!
70 finished!
71 finished!
72 finished!
73 finished!
74 finished!
75 finished!
76 finished!
77 finish

In [43]:
df_grouped.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


In [46]:
df_grouped.to_csv('geocoded df.csv')